In [1]:
## Imports
# Import the lsm.py file saved in the local directory:
from cases.lsm.lsm import *

# Python std. lib data handling
import os
from pathlib import Path

# Note: this import for extract_and_process_interface_points needs to be modified so that it
#       it matches where you saved the code that you supplied to us.
from data.data_levelset_geometric_shapes.extract_and_process_interface_points import read_data


### Data imports

In this cell we import the extracted LSM intersection point data:

In [2]:
# NOTE: Change the string below to where your data is stored:
filename = Path("../../ddgclib/data_levelset_geometric_shapes/sphere_coarse/extraction_data_0.000000.txt")
result_folder = Path("../../ddgclib/data_levelset_geometric_shapes/X_intersections_sphere_coarse")
plot_single_cells = False

# Create the result folder
if not os.path.exists(result_folder):
    os.mkdir(result_folder)
    
# Read and plot the data
corners_and_intersections = read_data(filename)
#unique_intersections = extract_and_save_unique_intersection_points(corners_and_intersections, result_folder)
#plot_and_save_levelset_points(corners_and_intersections, result_folder)
#plot_unique_intersections(unique_intersections, result_folder)
#plot_intersection_points(corners_and_intersections, result_folder, plot_single_cells)

cell_corners_and_intersections = corners_and_intersections
def intersection_is_found(intersection, ref_intersections):
    return any([all([np.abs(coord - coord_ref) <= 1e-14 for coord, coord_ref in zip(intersection, ref_inter)]) for ref_inter in ref_intersections])
    # Get unique intersections
all_intersections    = [inter for data in cell_corners_and_intersections for inter in data["Intersections"]]
unique_intersections = np.unique(np.array(all_intersections), axis=0)


Total number of cells          :  128
Total number of intersections  :  504
Total number of levelset points:  128


# Compute curvature locally

Below we demonstrate how to compute for the first cell in the index list `cell_i = 0`.
The `cell_corners_and_intersections` list should be a list that contains all cells around `cell_i` which have any intersection point. 

In [3]:
# Find the contributions that need to be added here to each cell in the current vertex
# use the edge
cell_i = 0
hndA_i = curvature_cell_i(cell_i, cell_corners_and_intersections)
hndA_i

array([0.16851428, 0.08325927, 0.01390179])

In [4]:
n = len(cell_corners_and_intersections)  # Total number of cells for which to 
int_hndA_i = 0
for i in range(n):
    cell_i = i
    hndA_i = curvature_cell_i(cell_i, cell_corners_and_intersections)
    int_hndA_i += 2*hndA_i
    
int_hndA_i

array([-0.15566344,  0.15566344, -0.84508736])

In [6]:
r = 0.15 # Radius of the tube (20 mm)
theta_p = 20 * np.pi/180.0  # Three phase contact angle
r = np.array(r, dtype=np.longdouble)

2*np.linalg.norm(int_hndA_i), 4*np.pi*r

(1.7465793496428834, 1.8849555921538757999)

# Compute total curvature of test case
### Coarse refinement

NOTE: It is not recommended to use the for loop below as this recomputes all the curvatures every time, this is for quick demonstration purposes only. `cell_corners_and_intersections

In [7]:
n = len(cell_corners_and_intersections)  # Total number of cells for which to 
int_hndA_i = 0
for i in range(n):
    cell_i = i
    hndA_i = curvature_cell_i(cell_i, cell_corners_and_intersections)
    int_hndA_i += 2*hndA_i
    
int_hndA_i

array([-0.15566344,  0.15566344, -0.84508736])

### Medium refinement

In [9]:
# NOTE: Change the string below to where your data is stored:
filename = Path("../../ddgclib/data_levelset_geometric_shapes/sphere_medium/extraction_data_0.000000.txt")
plot_single_cells = False
   
# Read and plot the data
corners_and_intersections = read_data(filename)

cell_corners_and_intersections = corners_and_intersections
def intersection_is_found(intersection, ref_intersections):
    return any([all([np.abs(coord - coord_ref) <= 1e-14 for coord, coord_ref in zip(intersection, ref_inter)]) for ref_inter in ref_intersections])
    # Get unique intersections
all_intersections    = [inter for data in cell_corners_and_intersections for inter in data["Intersections"]]
unique_intersections = np.unique(np.array(all_intersections), axis=0)


Total number of cells          :  416
Total number of intersections  :  1656
Total number of levelset points:  416


In [10]:
n = len(cell_corners_and_intersections)  # Total number of cells for which to 
int_hndA_i = 0
for i in range(n):
    cell_i = i
    hndA_i = curvature_cell_i(cell_i, cell_corners_and_intersections)
    int_hndA_i += 2*hndA_i
    
int_hndA_i

array([ 0.00737443,  0.04562704, -0.17994765])

### Fine refinement

In [11]:
# NOTE: Change the string below to where your data is stored:
filename = Path("../../ddgclib/data_levelset_geometric_shapes/sphere_fine/extraction_data_0.000000.txt")
plot_single_cells = False
   
# Read and plot the data
corners_and_intersections = read_data(filename)

cell_corners_and_intersections = corners_and_intersections
def intersection_is_found(intersection, ref_intersections):
    return any([all([np.abs(coord - coord_ref) <= 1e-14 for coord, coord_ref in zip(intersection, ref_inter)]) for ref_inter in ref_intersections])
    # Get unique intersections
all_intersections    = [inter for data in cell_corners_and_intersections for inter in data["Intersections"]]
unique_intersections = np.unique(np.array(all_intersections), axis=0)


Total number of cells          :  1760
Total number of intersections  :  7032
Total number of levelset points:  1760


In [12]:
n = len(cell_corners_and_intersections)  # Total number of cells for which to 
int_hndA_i = 0
for i in range(n):
    cell_i = i
    hndA_i = curvature_cell_i(cell_i, cell_corners_and_intersections)
    int_hndA_i += 2*hndA_i
    
int_hndA_i

MemoryError: Unable to allocate 121. GiB for an array with shape (1758, 1758, 1758, 3) and data type float64

#### Comparison with scalar analytical solution

The sphere has a radius of $r = 0.15$ m and therefore a point-wise mean ("fluid mean") curvature of $H = \frac{2}{r}$  and an area of $A = 4\pi r^2$

$$ \int H N dA = \frac{2}{r} \cdot 4\pi r^2 = 8 \pi r  $$

In [8]:
r = 0.15 # Radius of the tube (20 mm)
theta_p = 20 * np.pi/180.0  # Three phase contact angle
r = np.array(r, dtype=np.longdouble)

2*np.linalg.norm(int_hndA_i), 4*np.pi*r

(1.7465793496428834, 1.8849555921538757999)

In [15]:
r = 0.15 # Radius of the tube (20 mm)
theta_p = 20 * np.pi/180.0  # Three phase contact angle
r = np.array(r, dtype=np.longdouble)

2*np.linalg.norm(np.array([ 0.00737443,  0.04562704, -0.17994765])), 4*np.pi*r

(0.3715769946458419, 1.8849555921538757999)